In [ ]:
%qtconsole

In [ ]:
import numpy as np
from scipy.fftpack import fft, ifft, fftfreq
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.misc import comb
dB = lambda x: 20*np.log10(np.abs(x))

In [ ]:
fs = 1e6
nfft = 2**20
t = np.arange(n)/fs
f = fftfreq(nfft,1/fs)
tpi = 2*np.pi

sTrans = np.sin(tpi*t*1e3)

Define the nonlinearity as a polynomial p with even orders, try to indentify the coefficients from the difference tones, and find the correct predistortion for the modulating signal.

In [ ]:
pActual = np.poly1d([0.45,0,0.11,0,0.8,0,0.3,0,0.777,1,0])
#pActual = np.poly1d([1,0,1,1,0])
#pActual = np.poly1d([0.45,0,0.11,0,0,0,0,0,0,1,0])
df=f[1]
f0 = (40e3//df)*df
f1 = (41e3//df)*df
harmonics = pActual.order//2 # In the audible range, corresponds to even order nonlinearities

sIn = np.sin(tpi * t * f0) + np.sin(tpi * t * f1)
#sIn /= np.max(np.abs(sIn))
sOut = pActual(sIn)+sIn
SOut = fft(sOut)/nfft
fDiff = np.abs(f0-f1)
levels = np.zeros(harmonics,dtype=complex)
def levelAtFrequency(Sig,fCenter,fWidth):
    idx = (fCenter-fWidth < f) & (f < fCenter+fWidth)
    inRange = Sig[idx]
    maxIdx = np.argmax( np.abs(inRange) ) 
    #return np.abs(inRange[maxIdx])
    return inRange[maxIdx]
refLevel = levelAtFrequency(SOut,f0,fDiff/5)

for n in range(harmonics):
    levels[n] = levelAtFrequency(SOut,(n+1)*fDiff,fDiff/5)/refLevel

def audibleLevelFactor(n,h):
    if h<1 or 2*h>n or n%2 is not 0:
        return 0
    else:
        return 2/2**n*comb(n,n//2)*comb(n,n//2-h)
    
pIdent = np.poly1d([1j]) # Needed to keep the polynomial as complex
for n in range(harmonics,0,-1):
    #print('n: {}'.format(n))
    higherContrib = 0
    for m in range(n+1,harmonics+1):
        higherContrib += pIdent[2*m]*audibleLevelFactor(2*m,n)
        #print('\t m: {:.2f}\t pIdent[2m]: {:.2f}\t audFact(2m,n): {:.2f}\t higherCont: {:.2f}'.
        #      format(m,pIdent[2*m],audibleLevelFactor(2*m,n),higherContrib))
    pIdent[2*n] = (levels[n-1]-higherContrib)/audibleLevelFactor(2*n,n)

pIdent -= 1j
pIdent /= np.max(np.abs(pIdent))

print('Actual nonlinearity is \n{}'.format(pActual))
print('Identified nonlinearity is \n{}'.format(pIdent))

Note that the factor in the highest level audible harmonic for a given order (n,n/2) is the same factor that should be used for the polynomial to invert! This can be used as a justification somehow.

In [ ]:
def findToInvert(pActual):
    pToInvert = np.poly1d([])
    fraction=2
    for n in range(2,pActual.order+1,2): # +1 to get inclusive
        fraction*= (n-1)/n
        pToInvert[n] = pActual[n]*fraction
    return pToInvert

#pActual = np.poly1d([1,0,1,0,1,0])
#pToInvert = np.poly1d([0.4921875,0,0.546875,0,0.625,0,0.75,0,1,0,0])


pToInvert = findToInvert(pActual)
print('Actual nonlinearity is \n{}'.format(pActual))
print('ToInverting nonlinearity is is \n{}'.format(pToInvert))
x = np.linspace(0,2,10000)
y = pToInvert(x)
pInv = interp1d(y,x)
#pinv = lambda x: x**0.5
win = signal.windows.tukey(n,0.1)
env = pInv(1+0.95*sTrans)
mod = env*np.sin(tpi*t*40e3)
#mod = np.real( np.exp(signal.hilbert(np.log(env))) * np.exp(1j*tpi*t*40e3) )

demod = pActual(mod)
Demod = fft(demod)/n
#Demod /= np.max(np.abs(Demod))


In [ ]:
fMin = 1e3
fMax = 5e3
exclude = np.r_[39e3,40e3,41e3]
#exclude = np.r_[1e3]

idx = (f>fMin-0.5e3) & (f<fMax+0.5e3)
plt.plot(f[idx],dB(Demod[idx]))
plt.show()

THD = 0
for fc in np.arange(fMin,fMax+1,1e3):
    if  any( (ex>fc-0.2e3) & (ex<fc+0.2e3) for ex in exclude):
        continue
    #print(fc)
    idx = (f>fc-0.2e3) & (f<fc+0.2e3)
    THD += np.max( np.abs(Demod[idx]))
fc=40e3
idx = (f>fc-0.2e3) & (f<fc+0.2e3)
THD /= np.max( np.abs(Demod[idx]))
print('THD: {:.2e}'.format(THD))

